### Importing Libraries

In [4]:
import tensorflow as tf 
import keras 
import numpy as np
from keras.models import Model
from keras.layers import Layer
from keras.layers import (Reshape, Conv2DTranspose, Add, Conv2D, MaxPool2D,
                          Flatten, InputLayer, BatchNormalization, Input, )
from keras.optimizers import Adam 

### Data Preparation

In [5]:
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
mnist_digits = np.concatenate([x_train, x_test], axis = 0)
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [6]:
dataset = tf.data.Dataset.from_tensor_slices(mnist_digits)

In [7]:
len(dataset)

70000

In [8]:
batch_size = 128

In [9]:
train_dataset = (
    dataset.shuffle(buffer_size=1024, reshuffle_each_iteration=True)
    .batch(batch_size).prefetch(tf.data.AUTOTUNE)
)

In [10]:
train_dataset

<_PrefetchDataset element_spec=TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None)>